# #201 Beams with mass
<i>Comparison of adding additional mass</i>
***

In [ ]:
# Input Params
length = 5.0 #m		     Length of beam
element_length = 1
beam_z_spacings = [0, 2, 4, 6, 8, 10]    # No Mass, Global L2M, Beam L2M, 2xDensity, Mass Factor, Mass Elements

# Beam Mass
concrete_mass = 2.5 # tonnes/m^3
beam_width = 0.3 # m
beam_depth = 0.5 # m
beam_mass_per_metre =  concrete_mass * (beam_width*beam_depth)

# Additional Mass (double existing beam)
mass_material_per_metre = beam_mass_per_metre # tonnes/metre 
mass_modification_factor = 2.0
# Equivalent Loading /m
equivalent_mass_loading = mass_material_per_metre*9.81

# Concrete stiffness E
Ec = 35_000_000
Iyy = beam_width*beam_depth**3/12

In [534]:
print("Beam mass t/m = ", beam_mass_per_metre)
print("Beam load kN/m = ", equivalent_mass_loading)

Beam mass t/m =  0.375
Beam load kN/m =  3.67875


In [535]:
import math
mode1_beam = 9.87/(2*math.pi) * math.sqrt((Ec*Iyy*9.81)/(equivalent_mass_loading*length**4))
mode1_mass = 9.87/(2*math.pi) * math.sqrt((Ec*Iyy*9.81)/(2*equivalent_mass_loading*length**4))

print(f"Expected frequency mode 1                      = {mode1_beam:.4f}Hz")
print(f"Expected frequency mode 1 with additional mass = {mode1_mass:.4f}Hz")

Expected frequency mode 1                      = 33.9344Hz
Expected frequency mode 1 with additional mass = 23.9953Hz


#### Connect to LUSAS and create a new model database

In [536]:
import sys; sys.path.append('../') # Reference modules in parent directory
from LPI_22_0 import *
lusas = get_lusas_modeller()
# if lusas.existsDatabase():
#     raise Exception("This script will create a new model. Please save and close the current model and try again")

import m100_Tools_And_Helpers.Helpers as Helpers
Helpers.initialise(lusas)

In [537]:
# Create a new model
lusas.newProject("Structural", "Beams_with_mass.mdl")
# Get a reference to the model database
db = lusas.getDatabase()
# Set the vertical axis
db.setAnalysisCategory("3D")
db.setVerticalDir("Z")
# Set the unit system
db.setModelUnits("kN,m,t,s,C")

### Create model geometry

In [538]:
# Create a beam lines
beams = []
for z in beam_z_spacings:
    beams.append(Helpers.create_line([0,0,z], [length, 0, z]))

# Create a duplicate line to hold mass elements over the last line
db.options().setBoolean("newFeaturesMergeable", False)
mass_line = Helpers.create_line([0,0,beam_z_spacings[-1]], [length, 0, beam_z_spacings[-1]])

### Create model attributes

In [539]:
# Create a beam mesh attribute
beam_mesh_attr = db.createMeshLine("Beam Mesh")
# Set the element type and size of elements BMI21 = 2Node thick beam elements, 1 = 1m length
beam_mesh_attr.setSize("BMI21", element_length)

# Create a Non-Structural Mass mesh attribute
mass_mesh_attr = db.createMeshLine("Non-Structural Mass Mesh")
mass_mesh_attr.setSize("LMS3", element_length)

# Create a equivalence mesh attribute, to combine the beam and mass meshes
equivalence_mesh_attr = db.createEquivalence("Mass mesh equivalence").setEquivalence(1e-1, True)

In [540]:
# Create a geometric attribute
geometric_attr = Helpers.create_rectangular_section(db, "Rectangular Section", 0.3, 0.5)

In [541]:
# Create a linear elastic material attribute
material_attr = db.createIsotropicMaterial(f"Concrete", Ec, 0.2, concrete_mass, 10e-6, 2.0)
material_attr.setDescription(f"{concrete_mass=}(t/m^3)")

material_double_mass_attr = db.createIsotropicMaterial(f"Concrete double mass", Ec, 0.2, 2*concrete_mass, 10e-6, 2.0)
material_double_mass_attr.setDescription(f"concrete_mass={2*concrete_mass}(t/m^3)")

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFAttribute instance at 0x2373920488928>

In [542]:
# Create a mass material attribute
mass_material_attr = db.create3dMassMaterial(f"Mass material", mass_material_per_metre, mass_material_per_metre, mass_material_per_metre, "line")
mass_material_attr.setDescription(f"{mass_material_per_metre=:.4f}(t/m)")

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFAttribute instance at 0x2374003750176>

In [543]:
''' Create a section property modifier attribute'''
section_property_modifier_attr = db.createSectionPropertyModifier(f"Mass modification factor")
section_property_modifier_attr.setMassFactor("A", mass_modification_factor)
section_property_modifier_attr.setDescription(f"{mass_modification_factor=}")

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFAttribute instance at 0x2374003680240>

In [544]:
''' Create support attributes '''
# Freedoms F=Free, R=Restrained, S=Spring
pinned_support_attr = db.createSupportStructural("Pinned")
pinned_support_attr.setStructural("R", "R", "R", "R", "F", "R", "F", "F", "F")

slide_support_attr = db.createSupportStructural("Slide")
slide_support_attr.setStructural("F", "R", "R", "R", "F", "R", "F", "F", "F")

# This will be used to effectively remove beams from eigen analyses
fully_fixed_support_attr = db.createSupportStructural("Fully Fixed")
fully_fixed_support_attr.setStructural("R", "R", "R", "R", "R", "R", "F", "F", "F")

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFSupportStructural instance at 0x2373921015840>

In [545]:
# Beam distributed loading
beam_distributed_load_attr = db.createLoadingBeamDistributed(f"Beam distributed load")
beam_distributed_load_attr.setBeamDistributed("Parametric", "Global", "beam")
beam_distributed_load_attr.addRow(0.0, 0.0, 0.0, -equivalent_mass_loading, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, -equivalent_mass_loading, 0.0, 0.0, 0.0)
beam_distributed_load_attr.setDescription(f"{equivalent_mass_loading=:.4f}(kN/m)")
# Global distributed loading
global_distributed_load_attr = db.createLoadingGlobalDistributed(f"Global distributed load")
global_distributed_load_attr.setGlobalDistributed("Length", 0.0, 0.0, -equivalent_mass_loading, 0.0, 0.0, 0.0, 0.0, 0.0, False, 0.0)
global_distributed_load_attr.setDescription(f"{equivalent_mass_loading=:.4f}(kN/m)")

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFAttribute instance at 0x2374004063952>

In [546]:
# Design attributes for identification
design_attributes = []
for i, id in enumerate(["Glbl L2M", "Beam L2M", "2xDensity", "Mass Factor", "Mass Elements"]):
    attr = db.createDesignAttribute(id, "MassType", "MassType", "Lines")
    attr.assignTo(beams[i+1])

### Assign attributes

In [547]:
''' Assign the attributes to the geometry '''
# get the assignment object
assignment = lusas.assignment().setAllDefaults()
# Assign the mesh
beam_mesh_attr.assignTo(beams, assignment)
# Assign the geometry
geometric_attr.assignTo(beams, assignment)
# Assign the material
material_attr.assignTo(beams, assignment)

# Special mass elements
mass_mesh_attr.assignTo(mass_line, assignment)
mass_material_attr.assignTo(mass_line, assignment)
# Equivalence the mass and beam meshes of teh last beam
equivalence_mesh_attr.assignTo(mass_line, assignment)
equivalence_mesh_attr.assignTo(beams[-1], assignment)

# Mass Modifier
section_property_modifier_attr.assignTo(beams[-2])

# 2x Mass Material
material_double_mass_attr.assignTo(beams[-3])

# Loads to mass
global_distributed_load_attr.assignTo(beams[-4], assignment)
beam_distributed_load_attr.assignTo(beams[-5], assignment)

# Assign the supports to the points of the line
for line in beams:
    pinned_support_attr.assignTo(line.getStartPoint(), assignment)
    slide_support_attr.assignTo(line.getEndPoint(), assignment)

### Create loadcases and analyses

Linear Static Analysis

In [548]:
linear_static_analysis = db.getAnalysis("Analysis 1")
linear_static_analysis.setName("00 Linear Static")
# Rename the loadcase
first_loadcase = Helpers.get_loadcase(db, 1)
first_loadcase.setName("Gravity")
first_loadcase.addGravity(True)

Helper methods for creation of eigenvalue frequency analyses with and without loading to mass conversion

In [549]:
def create_eigen_anaysis(name:str, loadcase_name:str, no_modes=2) -> IFLoadcase:
    # Create a new analysis without an initial loadcase
    db.createAnalysisStructural(name, False)
    first_loadcase = db.createLoadcase(loadcase_name, name)
    # Set the loadcase controls to be a frequency analysis searching for the minimum no_modes natural frequencies
    first_loadcase.setEigenvalueMaxMinControl("Frequency", "Minimum", no_modes)
    # Set the eiegenvector normalisation procedure to mass for a frequency analysis - This just scales the resulting eigenvector and is the default setting
    first_loadcase.getEigenvalueControl().setValue("NormalisationProcedure", "GlobalMass")
    return first_loadcase

In [550]:
def create_eigen_anaysis_load_to_mass(name:str, loadcase_name:str, no_modes=2) -> IFLoadcase:
    # Create a new analysis without an initial loadcase
    db.createAnalysisStructural(name, False)
    first_loadcase = db.createLoadcase(loadcase_name, name)
    # Set the loadcase controls to be a frequency analysis searching for the minimum no_modes natural frequencies
    first_loadcase.setEigenvalueMaxMinControl("Frequency", "Minimum", no_modes)
    # Set the eiegenvector normalisation procedure to mass for a frequency analysis - This just scales the resulting eigenvector and is the default setting
    first_loadcase.getEigenvalueControl().setValue("NormalisationProcedure", "GlobalMass")
    # Consider the assigned loading as mass in the eigenvalue analysis
    first_loadcase.getEigenvalueControl().setValue("loadToMass", True)
    return first_loadcase

In [551]:
for i, beam in enumerate(beams):
    loadcase = create_eigen_anaysis(f"0{i+1} Eig freq", f"Beam {i+1}")
    assignment = lusas.assignment().setAllDefaults().setLoadset(loadcase)
    fully_fixed_support_attr.assignTo(beams[:i]+beams[i+1:], assignment)

In [552]:
for i in [1,2]:
    loadcase = create_eigen_anaysis_load_to_mass(f"1{i+1} Eig freq L2M", f"Beam-{i+1} (L2M)")
    assignment = lusas.assignment().setAllDefaults().setLoadset(loadcase)
    fully_fixed_support_attr.assignTo(beams[:i]+beams[i+1:], assignment)
    global_distributed_load_attr.assignTo(beams[1], assignment)
    beam_distributed_load_attr.assignTo(beams[2], assignment)

### Mesh the model

In [553]:
db.resetMesh()
db.updateMesh()

<win32com.gen_py.LUSAS Modeller ActiveX Script Language 22.0.IFDatabase instance at 0x2374003810432>

# Solve and open results

In [554]:
for analysis in db.getAnalyses():
    analysis.solve(True)
db.openAllResults(False)

In [ ]:
lusas.view().setFromAxis("+X+Z")

lusas.view().insertDiagramsLayer()
lusas.view().diagrams().setResults("Force/Moment - Thick 3D Beam", "My")
lusas.view().diagrams().setLocation("Gauss")
lusas.view().diagrams().setShowDeformed(True)

lusas.view().insertValuesLayer()
lusas.view().values().setResults("Reaction", "FZ")

lusas.view().geometry().autoColourByAttributes("Design.MassType", True)
lusas.view().insertAnnotationLayer()

# lusas.view().showCoordSystem(False)
# lusas.view().attributes().visualiseNone("Supports")

lusas.view().insertContoursLayer()
lusas.view().contours().setResultsTransformNone()
lusas.view().contours().setResults("Displacement", "DZ")
lusas.view().contours().chooseSettings(1)
lusas.view().contours().setShowDeformed(True)

lusas.view().attributes().setShowDeformed("Geometric", True)

lusas.view().showViewSummary(True)
